# **install libs**

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


# **Import Neccessary lib**

In [ ]:
from transformers import pipeline ,set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
from evaluate import load
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
import torch
from tqdm import tqdm
import pandas


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Model and Tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# **Split the dataset into chunks of data**

In [ ]:
def batch_of_chunks(dataset_columns,batch_size):

  for i in range(0, len(dataset_columns),batch_size):

    yield dataset_columns[i : i + batch_size]


# **Calculate_Metrics**

In [ ]:
def calculate_metric(dataset, metric, model, tokenizer, batch_size = 16, device = device, column_text = 'dialogue' , column_summary = 'summary'):

  article_batches = list(batch_of_chunks(dataset[column_text],batch_size))
  target_batches = list(batch_of_chunks(dataset[column_summary],batch_size))

  for article_batch,target_batch in tqdm(zip(article_batches,target_batches),total = len(article_batches)):

    inputs = tokenizer(article_batch,max_length=1024,truncation=True,padding='max_length',return_tensors='pt')

    summary =  model.generate(input_ids = inputs['input_ids'].to(device),
                             attention_mask = inputs['attention_mask'].to(device),
                             length_penalty = 0.8, num_beams = 8, max_length = 128)


    decode_summary = [tokenizer.decode(s,skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summary]

    metric.add_batch(predictions = decode_summary , references = target_batch)

  score = metric.compute()
  return score


# **Load Datset**

In [ ]:
dataset_samsum = load_dataset('samsum')

split_length = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f'Split Length : {split_length}')
print(f'Features : {dataset_samsum["train"].column_names}')

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Split Length : [14732, 819, 818]
Features : ['id', 'dialogue', 'summary']


# **Pipeline**

In [ ]:
pipe = pipeline('summarization',model = model_ckpt)

res = pipe(dataset_samsum['test'][0]['dialogue'])
print(res)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


[{'summary_text': "Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him ."}]


# **Load Metric**

In [ ]:
rouge_metric = load('rouge')

# **Score**

In [ ]:
score = calculate_metric(dataset_samsum['test'],rouge_metric,model_pegasus,tokenizer,batch_size = 2,column_text = 'dialogue',column_summary = 'summary')


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

# **Train Arguments and Trainer**

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainerArguments(
    output_dir = 'pegasus-samsum',num_train_epochs = 1, warmup_steps = 500,
    per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
    weight_decay = 0.01, logging_steps = 10,
    evaluation_strategy = 'steps', eval_steps = 500, save_steps = 1e6,
    gradient_accumulation_steps = 16
)

In [ ]:
trainer = Trainer(model = model_pegasus,
                  args = trainer_args, tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt['train'],
                  eval_dataset = dataset_samsum_pt['validation'])

In [ ]:
trainer.train()

# **Save the fine tuned model**

In [ ]:
model_pegasus.save_pretrained('pegasus-samsum-model')
tokenizer.save_pretrained('pegasus-samsum-model')


In [ ]:
dataset_samsum = load_dataset("samsum")

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

In [ ]:
sample_text = dataset_samsum["test"][0]["dialogue"]
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])